# Alignment workflow 
This notebook demonstrates the alignment workflow - possible combination of multiple alignment steps.

## Import libraries

In [ ]:
from align_panel.data_structure import ImageSetHolo
from align_panel.align.crop import CropAlignments
from align_panel.align.fine import FineAlignments
from align_panel.align.points import PointAlignments
from align_panel.image_transformer import ImageTransformer
import matplotlib
matplotlib.use('TkAgg') # solves problem with interactive matplotlib windows
import matplotlib.pyplot as plt

In [ ]:
path_1 ='/data/Hb-.dm3'
path_2 ='/data/Rb-.dm3'
path_3 ='/data/Hb+.dm3'
path_4 ='/data/Rb+.dm3'

In [ ]:
image_set1 = ImageSetHolo.load(path_1, path_2) # reference image set
image_set2 = ImageSetHolo.load(path_3, path_4) # image set to be aligned

In [ ]:
image_set1.phase_calculation()
image_set2.phase_calculation()

# ImageTransformer

Output of each transformation technique is an aligned image and transformation matrix. Because we need to combine matrices of several techniques, we need to store them in a list. For this purpose, we will use an `ImageTransformer` class. It takes an image as an input and can store multiple transformation matrices in its atribute. This class contains methods to calculate the final transformation matrix and to apply it to the image to obtain the aligned image.

In [ ]:
moving_image = ImageTransformer(image_set2.unwrapped_phase.data)

## First step of alignments - Crop and automatic alignments
All the alignment techniques have numpy arrays as an input for the images, therefore we need to obtain the numpy arrays from the `ImageSetHolo` instances. That is why `unwrapped_images.data` is called.

In [ ]:
crop_align = CropAlignments(image_set1.unwrapped_phase.data,image_set2.unwrapped_phase.data, method='euclidean')

Now, the result transformation matrix need to saved into the `moving_image` ImageTransformer.

In [ ]:
moving_image.add_transform(crop_align.tmat)

## Second step of alignments - Point alignments on cropped out parts

If the result of the alignments is not a desired one, the alignments can be called again. Other alignments can be called instead. When you are satisfied with the result, you can save the transformation matrix into the `moving_image` ImageTransformer with the `add_tranform` method.

In [ ]:
point_align = PointAlignments(crop_align._cropped_images['ref'], crop_align._cropped_images['mov'], method='euclidean',rebin = 8)

In [ ]:
moving_image.add_transform(point_align.tmat)

To get the result of the alignments from the `moving_image` ImageTransformer, you need to call the `get_transform-image()` method.

In [ ]:
plt.imshow(moving_image.get_transformed_image(), cmap='gray')
plt.show()